## Azimuthal Dust Profiles

In [ ]:
import numpy as np  
import matplotlib.pyplot as plt  
import astropy.constants as c  
import astropy.units as u  
%matplotlib inline

au = c.au.cgs.value

In [ ]:
import azimuthal_functions as az

### Set up model parameters of a radial gas profile

In [ ]:
na          = 100
ny          = 400 
A_gas       = 1.2    # gas density contrast (peak-to-valley)
sigma_y_deg = 10     # azimuthal bump extent in degree
d2g         = 1e-2
alpha       = 1e-3
v_frag      = 1000.
rho_s       = 1.67
M_star      = c.M_sun.cgs.value


sigma_y = sigma_y_deg / 180 * np.pi
r       = np.array([30 * au])                # radial grid
Y       = r[:, None] * np.linspace(- np.pi, np.pi, ny)  # azimuth grid
A       = np.logspace(-4, 0, na)             # particle size grid
sig_g   = 200 / (r / au)                     # avg. gas surface density
sig_d   = d2g * sig_g
T       = 200 * (r / au)**-0.5

sig_g_2D = 1 + (A_gas - 1) * np.exp(- Y**2 / (2 * (r * sigma_y))**2)
sig_g_2D *= sig_g / sig_g_2D.mean()

### Get the equilibrium dust distribution

In [ ]:
sig_d_2D = az.make_azim_distri(
    r,
    Y,
    A,
    T,
    sig_g_2D,
    sig_d,
    alpha,
    v_frag,
    rho_s,
    M_star,
)

### Plot the azimuthal distribution of grains

In [ ]:
f, ax = plt.subplots()

y = Y[0, :] / r * 180 / np.pi # azimuthal grid in degree

ax.plot(y, sig_g_2D[0, :] / 100, 'k-', label='gas / 100')

for ia in np.arange(na)[::na//6]:
    ax.plot(y, sig_d_2D[0, :, ia], 'x-', label=f'{A[ia]:.2g} cm')

ax.set_xlim(-45, 45)
ax.set_xlabel('azimuth [degree]')
ax.set_ylabel('$\Sigma_\mathrm{d}$ [g cm$^{-2}$]')
ax.legend();

### Plot the particle size distribution

In [ ]:
distri = sig_d_2D[0, :, :].mean(0)

f, ax = plt.subplots()

ax.loglog(A, distri)
ax.set_ylim(distri.max() * 1e-5, 2 * distri.max())
ax.set_xlabel('particle size [cm]')
ax.set_ylabel('$\sigma$ [g cm$^{-2}$]');

### Read opacity and interpolate on our grid

In [ ]:
lam_obs = 0.13 # our obervational wavelength

In [ ]:
import dsharp_opac as op
from scipy.interpolate import interp2d

In [ ]:
with np.load(op.get_datafile('default_opacities_smooth.npz')) as fid:
    a_opac   = fid['a']
    lam_opac = fid['lam']
    k_abs    = fid['k_abs']
    k_sca    = fid['k_sca']
    g        = fid['g']
    rho_s    = fid['rho_s']

f_kappa = interp2d(np.log10(lam_opac), np.log10(a_opac), np.log10(k_abs))
k_a = 10.**f_kappa(np.log10(lam_obs), np.log10(A))

In [ ]:
f, ax = plt.subplots()
ax.loglog(A, k_a)
ax.set_xlabel('particle size [cm]')
ax.set_ylabel('$\kappa_\mathrm{abs}$ [cm$^2$/g]')
ax.set_title(f'opacity at {10 * lam_obs:.2g} mm');

### Calculate intensity profile